# Story Generation based on a Given Prompt using DistilGpt-2

### Author : Kamlesh Kumar
### CMS : 348395
### Class : BSCS-10C
#### https://www.linkedin.com/in/kamlesh-kumar-389847224/

*installing additional libraries required*

In [7]:
%%capture
!pip install -q kaggle
!pip install -q transformers
!pip install accelerate -U
!pip install tensorboard

Text Generation Via Transformers:
Story Generation based on a Given Prompt

# Data Exploration:

In [2]:
# Store your Kaggle api file kaggle.json in the folder where this notebook is located

!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d ratthachat/writing-prompts

100%|███████████████████████████████████████▉| 369M/370M [00:38<00:00, 10.7MB/s]
100%|████████████████████████████████████████| 370M/370M [00:38<00:00, 10.1MB/s]


UsageError: Line magic function `%unzip` not found.


In [2]:
# Unzip the data writting prompts datset. 
!unzip writing-prompts.zip

Archive:  writing-prompts.zip
replace writingPrompts/README? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [5]:
# Set this variable to the path where you have stored the dataset

Dataset_path = "/home/kamlesh_kumar/FYP/GPT2Finetunning/writingPrompts/"
# !head {Dataset_path}train.wp_source
!head -n 7 {Dataset_path}train.wp_source

 You 've finally managed to discover the secret to immortality . Suddenly , Death appears before you , hands you a business card , and says , `` When you realize living forever sucks , call this number , I 've got a job offer for you . ''
 The moon is actually a giant egg , and it has just started to hatch .
 You find a rip in time walking through the alleys . You enter it to find yourself on a metal table with surgical instruments on a chair next to you .
 For years in your youth the same imaginary character appears in your dreams , you are good friends . Years later , when adult , you meet her in real life , she clearly recognises you and tries to avoid you , and you want answers .
 You glance at your watch 10:34 am , roughly 10 seconds later your plane explodes over the Pacific Ocean . Your eyes open as you jolt awake . The familiar hum of the planes engine remains . Checking your watch it is 9:35
 Through Iron And Flame
 You live in a world where there has never been sickness , and

In [7]:
# printing 4 lines of the target file i.e. the story
!head -n 1 {Dataset_path}train.wp_target

So many times have I walked on ruins , the remainings of places that I loved and got used to.. At first I was scared , each time I could feel my city , my current generation collapse , break into the black hole that thrives within it , I could feel humanity , the way I 'm able to feel my body.. After a few hundred years , the pattern became obvious , no longer the war and damage that would devastate me over and over again in the far past was effecting me so dominantly . <newline> It 's funny , but I felt as if after gaining what I desired so long , what I have lived for my entire life , only then , when I achieved immortality I started truly aging . <newline> <newline> 5 world wars have passed , and now they feel like a simple sickeness that would pass by every so often , I could no longer evaluate the individual human as a being of its own , the importance of mortals is merely the same as the importance of my skin cells ; They are a part of a mechanism so much more advanced , a mechan

In [16]:
import subprocess

# For removing the extra brackets from the source file. 
def remove_brackets_from_file(file_path):
    sed_command = r"sed -i 's/\[.*\]//g' {}"
    subprocess.call(sed_command.format(file_path), shell=True)

# For replacing some extra punctuations and spaces. 
def clean_punctuation(text):
    for p in '!,.:;?':
        text = text.replace(' ' + p, p)
    replacements = [
        (" n't", "n't"), (" 's", "'s"), (" 're", "'re"), (" 've", "'ve"),
        (" 'll", "'ll"), (" 'am", "'am"), (" 'm", "'m"), (" ' m", "'m"),
        (" 've", "'ve"), (" ' s", "'s"), ("<newline>", "\n")
    ]
    for old, new in replacements:
        text = text.replace(old, new)
    return text

def process_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    cleaned_lines = [clean_punctuation(line) for line in lines]
    with open(file_path, 'w') as file:
        file.writelines(cleaned_lines)

In [9]:
# Custom Dataset class for loading and encoding the dataset on fly, while training

from torch.utils.data import Dataset

class CustomDataset(Dataset):
    # def __init__(self, prompts_file, stories_file, tokenizer):
    #     self.prompts = open(prompts_file, 'r').readlines()
    #     self.stories = open(stories_file, 'r').readlines()
    #     self.tokenizer = tokenizer

    def __init__(self, prompts_file, stories_file, tokenizer, max_samples=30000):
        with open(prompts_file, 'r') as file:
            self.prompts = [next(file).strip() for _ in range(max_samples)]

        with open(stories_file, 'r') as file:
            self.stories = [next(file).strip() for _ in range(max_samples)]

        # Ensuring that both lists have the same length
        min_length = min(len(self.prompts), len(self.stories))
        self.prompts = self.prompts[:min_length]
        self.stories = self.stories[:min_length]

        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.prompts)

    def __getitem__(self, idx):
        prompt = self.prompts[idx].strip()
        story = self.stories[idx].strip()

        # Encode the prompt and story using the tokenizer
        combined_text = f"""prompt :
{prompt}
story: 
{story}
"""
        combined_text = clean_punctuation(combined_text)
        encoding = self.tokenizer(combined_text, return_tensors="pt", max_length=512, truncation=True)

        return {
            "input_ids": encoding["input_ids"].squeeze(),
            "attention_mask": encoding["attention_mask"].squeeze(),
        }

In [14]:
## Additional code for loading tokenized data directly into memory before start of training.

# class CustomDataset(Dataset):
#     def __init__(self, encoded_data):
#         self.encoded_data = encoded_data

#     def __len__(self):
#         return len(self.encoded_data)

#     def __getitem__(self, idx):
#         return self.encoded_data[idx]

# def encode_dataset(prompts_file, stories_file, tokenizer, max_length):
#     prompts = open(prompts_file, 'r').readlines()
#     stories = open(stories_file, 'r').readlines()
    
#     encoded_data = []
#     for prompt, story in zip(prompts, stories):
#         encoding = tokenizer(prompt.strip(), story.strip(), max_length=max_length, padding="max_length", truncation=True)
#         encoded_data.append({
#             "input_ids": encoding["input_ids"],
#             "attention_mask": encoding["attention_mask"],
#         })
    
#     return encoded_data

# Preprocessing

In [4]:
# Define your list of source and target file paths to make it easier to acess.

target_file_paths = [Dataset_path + "train.wp_target", Dataset_path + "valid.wp_target", Dataset_path + "test.wp_target"]

source_file_paths = [Dataset_path + "train.wp_source", Dataset_path + "valid.wp_source", Dataset_path + "test.wp_source"]

In [ ]:
## Code for encoding and loading the train and val dataset into memory.
 
# encoded_train_data = encode_dataset(source_file_paths[0], target_file_paths[0], tokenizer, max_length=512)
# encoded_val_data = encode_dataset(source_file_paths[1], target_file_paths[1], tokenizer, max_length=512)

# train_dataset = CustomDataset(encoded_train_data)
# val_dataset = CustomDataset(encoded_val_data)

In [8]:

# remove_brackets_from_file defined above

# Process each file
for file_path in source_file_paths:
    remove_brackets_from_file(file_path)
    # process_file(file_path)

print("Processing complete for all files.")


Processing complete for all files.


# Model Selection:
Model used is distilgpt2

In [6]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

model_name = "distilgpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name, pad_token='pad')
model = GPT2LMHeadModel.from_pretrained(model_name)

# tokenizer.pad_token = tokenizer.eos_token

from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False,
)
tokenizer.pad_token = tokenizer.eos_token

In [7]:
# moving model to GPU

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

# Fine tunning

In [13]:
!mkdir experiments

mkdir: cannot create directory ‘experiments’: File exists


In [14]:
# Creating the data loaders for training and validation dataset.
# Using 30000 samples for training and 10000 samples for validation

train_dataset = CustomDataset(source_file_paths[0], target_file_paths[0], tokenizer, max_samples = 30000)
val_dataset = CustomDataset(source_file_paths[1], target_file_paths[1], tokenizer, max_samples = 10000)

In [17]:
# testing dataset loader class.

output = train_dataset.__getitem__(5)

decoded_text = tokenizer.decode(output['input_ids'], skip_special_tokens=True)
print("\n\n\n")
print(decoded_text)

print("\n\n\n", output.keys())





prompt:
Through Iron And Flame
story: 
The mountain stood still and large beneath the Warrior. It had not trembled in the days since the people chose him as their Champion. `` Perhaps the Dragon slumbers yet, '' he thought. `` I shall wake him, soon enough. '' He strode forward, heavily, in his iron raiment, shining dully in the morning sun. The shrouded peak drew slowly nearer as the sun climbed the sky, mirroring the Warrior's toil this day as the last. The Dragon slept silently in its lair amidst the clouds. 
 
 For generations the people had not seen it wake. But their stories, told by night and hearth fire, still remained hushed in reverence of its deadly wrath. Legends told of it waking and the destruction that followed. But that was before. Before the people had tamed iron and bent it to their will. Now they had weapons that could fight the myth. That could quench the fire. 
 
 The Warrior stopped at the edge of the cloud bank, just as the sun had stopped there hours before.

In [18]:
train_dataset.__len__()

30000

In [19]:
# For setting up the trainer and training arguments

from transformers import Trainer, TrainingArguments

#setting up trainer arguments
training_args = TrainingArguments(
    output_dir="/home/kamlesh_kumar/FYP/GPT2Finetunning/experiments", #The output directory for saved model
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=10, # number of training epochs
    gradient_accumulation_steps=4,
    learning_rate=2e-3,
    lr_scheduler_type="cosine",
    save_total_limit=3,

    per_device_train_batch_size=8, # batch size for training
    per_device_eval_batch_size=8,  # batch size for evaluation
    eval_steps = 500, # Number of update steps between two evaluations.
    warmup_ratio=0.05,
    # fp16=True, #whether to use floating point 16 for training
    # fp16_opt_level="O1", #see apex AMP optimization level for detail
    # prediction_loss_only=True,

    logging_dir='/home/kamlesh_kumar/FYP/GPT2Finetunning/logs_dir',  # Directory for storing logs
    logging_strategy="steps",           # Log every 'logging_steps'
    logging_steps=500,                  # Log every 500 steps

    )

#instantiating a Trainer class object
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
    )

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


4 hours of training for 1 epoch and 8.2GBs of GPU.

In [ ]:
#conduct training
trainer.train()

/home/kamlesh_kumar/.venv/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


### Saving Model

In [ ]:
!mkdir final_model

In [ ]:
saved_model_path = "/home/kamlesh_kumar/FYP/GPT2Finetunning/final_model"
model.save_pretrained(saved_model_path)

# Evaluation

1. Results of tensorboard are analyzed in the report
2. Also I have added example for comparing two different models using human evaluation.
    1. model with 1 epoch on all the dataset
    2. model with 10 epoch with 30k examples.

In [1]:
!tensorboard --logdir=/home/kamlesh_kumar/FYP/GPT2Finetunning/logs_dir

TensorFlow installation not found - running with reduced feature set.
Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.15.1 at http://localhost:6006/ (Press CTRL+C to quit)
^C


# Story Generation or Inference

Stopping the prevoius code and inferencing by loading the both the pretrained and fie tunned models separately

In [1]:
# Inference on GPU

from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import set_seed    

# Load the fine-tuned model

# Path for model which has been trained for 1 epoch with 273000 examples
# saved_model_path = "/home/kamlesh_kumar/FYP/GPT2Finetunning/1_epoch_final_model"

# path for model which has been trained for 10 epochs with 30000 samples
saved_model_path = "/home/kamlesh_kumar/FYP/GPT2Finetunning/final_model"

pretrained_model = GPT2LMHeadModel.from_pretrained("distilgpt2")

# Load the model from the checkpoint
fine_tuned_model = GPT2LMHeadModel.from_pretrained(saved_model_path).to('cuda')

# Load tokenizer (same for both models)
tokenizer = GPT2Tokenizer.from_pretrained("distilgpt2", pad_token='pad')

tokenizer.pad_token = tokenizer.eos_token


def generate_text(model, prompt):
    model_inputs = tokenizer(prompt, return_tensors="pt").to('cuda')

#     # Generate text
#     output = model.generate(**encoding, max_new_tokens=512)
    sample_output = model.generate(
        **model_inputs,
        max_new_tokens=300,
        do_sample=True,
        top_k=0
    )

    return sample_output[0]

# prompt = """prompt :
# and how? :
# story:
# """

# # Generate text with pre-trained model
# # print("\nPre-trained Model:")
# # print(generate_text(pretrained_model, prompt))

# set seed to reproduce results. Feel free to change the seed though to get different results
set_seed(42)



In [2]:
## 1st story
prompt = """prompt :
You 've finally managed to discover the secret to immortality . Suddenly , Death appears before you , hands you a business card , and says , `` When you realize living forever sucks , call this number , I 've got a job offer for you . ''
story:
"""


output_ids_generation = generate_text(fine_tuned_model, prompt)

print("Output:\n" + 100 * '-')
print(tokenizer.decode( output_ids_generation , skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
prompt :
You've finally managed to discover the secret to immortality. Suddenly, Death appears before you, hands you a business card, and says, `` When you realize living forever sucks, call this number, I've got a job offer for you. ''
story:
`` you really aren't expecting this, are you? '' Death asked me quietly, not really caring if my answer was actually going to snowball. 
 
 `` I guess that means you're fucking *here*. '' I replied, a bit sheepish as I was trying to rationalize.Things like that usually happened when someone died, or some mundane at that moment suddenly different, in an instant or two. I was right, I was done. I had actually been offered the opportunity. 
 
 `` Yes. Well, I have actually got this deal for you, and as such you must understand I've lost quite a bit of your previous choice. Great deal. '' He started to shiver, and for a second I was filled wit

In [8]:
# 2nd story
prompt = """prompt :
You discover a hidden door in your basement that leads to an ancient library filled with unknown books.

story:
"""

output_ids_generation = generate_text(fine_tuned_model, prompt)

print("Output:\n" + 100 * '-')
print(tokenizer.decode( output_ids_generation , skip_special_tokens=False))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
prompt :
You discover a hidden door in your basement that leads to an ancient library filled with unknown books.

story:
I did it with confidence. 
 
 I ’ ve heard of this place before, the nice folks that live all over it, live by it ’ s gold. Nevertheless, I have several books unlaced within my basement. The first of them, which was lordly and elegant, contained 300 gold bottles. The second, the ones that have an ornately designed lock with two revolving doors. The third, and the fourth. Each filled with gold, containing what I can only guess was bolen when someone knocked on the door. The truths, as my brother and I learned in college, were written away in those sealed rooms. Some believe that that number spoke for free, others one would die of that theft. It is hard to tell and to have access to something close to books, not only by carefully looking in that order, but by sl

In [9]:
# 2nd story
prompt = """prompt :
Every night, you dream of a mysterious city. One day, you find a map leading to it in real life.

story:
"""

output_ids_generation = generate_text(fine_tuned_model, prompt)

print("Output:\n" + 100 * '-')
print(tokenizer.decode( output_ids_generation , skip_special_tokens=False))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
prompt :
Every night, you dream of a mysterious city. One day, you find a map leading to it in real life.

story:
A city kid rests outside his room playing video games in his room. To his surprise, the city has a map. From there, it starts to glow brighter and richer, until a certain explosion occurs when the player removes his glasses. The kid notices that one of his game characters is glowing brighter, and is able to take the city as a whole. He begins to yell around the room, with a mix of terror and frustration in his mind. As he goes around the room, more small explosions erupt in the city. Several people try to run down walls; some shout “ Medic! ” The kid screams, but his mom just blocks the walls, causing a woman to run and run over with a baby. Behind her, a bellowing amount of gunfire erupts, as lightning cracks in the city, and everyone just collapses out.
 
 When the

In [10]:
# 3rd story
prompt = """prompt :
You inherit an old watch that allows you to time travel, but only to moments of personal significance."

story:
"""

output_ids_generation = generate_text(fine_tuned_model, prompt)

print("Output:\n" + 100 * '-')
print(tokenizer.decode( output_ids_generation , skip_special_tokens=False))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
prompt :
You inherit an old watch that allows you to time travel, but only to moments of personal significance."

story:
I hate needles. At least I hate them. They're like chalk. 
 
 There are things that are freakin'in my stomach everyday. I've seen them like it on a coworkers'office supply market. Watch them bleed out to pieces, dye them- red and: just pieces of shit. It's not the freakin'misiest thing, and yeah, most certainly not with *this* goddamn standard watch. There are things that will never look past me. 
 
 You see, it's not the freakin'misest thing I vision, the exception just because the people with the bright purple watch you 'd've known are the true elite, the true elite, the true prize. 
 
 The problem is, they should all die right? It gets a little late though: wake up, there's hair on the body, still on the sheets. Tannhose is an understatement, those idiots s

In [11]:
# 4th story
prompt = """prompt :
One morning, you wake up in a world where everyone can hear each other's thoughts.

story:
"""

output_ids_generation = generate_text(fine_tuned_model, prompt)

print("Output:\n" + 100 * '-')
print(tokenizer.decode( output_ids_generation , skip_special_tokens=False))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
prompt :
One morning, you wake up in a world where everyone can hear each other's thoughts.

story:
What the fuck kind of person is this? Everybody is so fucking goddamn gullible that they can see the color streamed from their phones while I was eating my cereal, and this is the only time I live in my apartment, so I had to call Diane over to make sure everything went fine, and no good lies are about to spread across the world before you try to tell them to stay away from me. I'll tell you what you realize is my greatest fear. 
 
 As I walked out of the door I saw you all dolled up to the sky. 
 
 I saw no one there, but I sluggishly looked over and saw you all dolled up, because what? Who are you and what's your name? I guess I'm about 12 or ''. My girlfriend's been dead for two weeks... no ONE said she 'd seen me fall to her death, like everyone else. 
 
 Still, enough of thes

In [12]:
# 5th story
prompt = """prompt :
A mysterious figure leaves a puzzle box on your doorstep every year on your birthday.

story:
"""

output_ids_generation = generate_text(fine_tuned_model, prompt)

print("Output:\n" + 100 * '-')
print(tokenizer.decode( output_ids_generation , skip_special_tokens=False))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
prompt :
A mysterious figure leaves a puzzle box on your doorstep every year on your birthday.

story:
The presents arrived precisely at 11:00 this day. I no longer heard Mr. Christopher Tolkien himself talk excitedly after saving the day. I was turned to the side because I didn't know if he knew I would wake up this morning with an actual skeleton in my closet. Unaware of his actions, I had opened the box to curiosity and addressed it to whoever I was supposed to be dead. 
 
 `` Christopher... um... my birthday... am I dead? '' 
 
 I've got a few confused looks as I read the newspaper. `` MIREARY NEEDED TO BE FREE!!! '' 
 
 Wadday then? This is certainly not a real birthday wish. I could wish for unlimited knowledge for everyone. Wouldn't he know I didn't wish for unlimited knowledge? What do I know now? What does that have to do with me? 
 
 Mr. Christopher Tolkien seems to ha

In [13]:
# 6th story
prompt = """prompt :
You find a strange plant in your garden that grows objects instead of flowers.

story:
"""

output_ids_generation = generate_text(fine_tuned_model, prompt)

print("Output:\n" + 100 * '-')
print(tokenizer.decode( output_ids_generation , skip_special_tokens=False))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
prompt :
You find a strange plant in your garden that grows objects instead of flowers.

story:
What bugger, what am I going to do with anything anyway? The back--style clock reminding me of quitting early and the sooner it comes to 8am already I can tell the neighbours what is going on. The neighbours counter, stocked up with alcohol and the nearest dog to the door with 8 am standing patiently, patiently waiting to be sniffed up and walked off with the rest of the empty sky. 
 
 The empty spot on the front garden was alarming, wide enough for the whole neighbourhood to sit in. I had installed something along the market in an attempt to add to my monthly squabble, and while it would have come to an end why not, a sip of lemonade was going to help and help fill the house with the energy needed for the evening. 
 
 It was always the case that things got messy when I got rid of som

In [14]:
# 7th story
prompt = """prompt :
A diary from the future falls into your hands, describing events yet to happen in your life..

story:
"""

output_ids_generation = generate_text(fine_tuned_model, prompt)

print("Output:\n" + 100 * '-')
print(tokenizer.decode( output_ids_generation , skip_special_tokens=False))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
prompt :
A diary from the future falls into your hands, describing events yet to happen in your life..

story:
I wake up now, confused and exhausted. Emergency call lead to proportions of pockets, coins and other odds. Gangs greet me from the doorway, they yell at me as if it is not the right way in the world. I reach into my pockets to find my purse. Is there something inside me? I hesitate, I left it in my pocket somewhere to get a sewing kit and now it contains what I consider to be an ordinary circular object. I feel confused, the components have become unimportant, such as a pocket watch. An old part of me has been spending the last 18 years on the casing for years. Whenever I was of age I still remember. We joked, made jokes, made conversations. We drank coffee, made love. We sat at the pastel beds, people around us laughed and talked. We made love. We kicked young men unt

In [15]:
# 8th story
prompt = """prompt :
You receive a letter from your future self warning you about a decision you're about to make.
story:
"""

output_ids_generation = generate_text(fine_tuned_model, prompt)

print("Output:\n" + 100 * '-')
print(tokenizer.decode( output_ids_generation , skip_special_tokens=False))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
prompt :
You receive a letter from your future self warning you about a decision you're about to make.
story:
A change of clothing is the most important decision, to make sure your `` Jew '' speech is not actually over. This beautiful woman has made her experience known. She has to wear a special dress for a special of a special occasion. Her favorite piece of clothing is the one that hints at the physical beauty of a person's appearance. It is: a burgundy red dress that reveals the choices of manufacturing components having been purchased for months. 
 
 She has to wear a specially fitted suit that would fit the description of any gifted person in the world. It is ridiculous, she states. Her voice sounds monotonous.. unless used in a heard from her vicinity, she will say something like: 
 
 `` Please absolve your sins with whatever weapon you can find on earth. If you will like

In [16]:
# 9th story
prompt = """prompt :
The moon is actually a giant egg , and it has just started to hatch .
story:
"""

output_ids_generation = generate_text(fine_tuned_model, prompt)

print("Output:\n" + 100 * '-')
print(tokenizer.decode( output_ids_generation , skip_special_tokens=False))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
prompt :
The moon is actually a giant egg, and it has just started to hatch.
story:
An oily click clings to the chirp as Gene slams shutters his funeral parlor. Dave screams across the concrete floor, his breath rattling in his ears. 
 
 `` Fucking tin foil. '' Gene mutters. He turns around and stares at Risley, the last of his kind maintained seated at the foot of his bed, watching closely as Risley absorbs the egg into his hands. 
 
 Risley adjusts her collar for reassurance she doesn't storm in and when she does the dreaded silence of the funeral she jumps unceremoniously. 
 
 watches Ollivander shuffle over to the table and Risley begins to sob, rubbing his eyes. Her death cries above her become desperate screams. 
 
 Risley puts on her best red startle and backflips Risley puts her fingers up to her throat. She hears a clattering of metal and wood. Her eyes are just as stun

In [18]:
# 10th story
prompt = """prompt :
You discover you can communicate with animals.
story:
"""

output_ids_generation = generate_text(fine_tuned_model, prompt)

print("Output:\n" + 100 * '-')
print(tokenizer.decode( output_ids_generation , skip_special_tokens=False))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
prompt :
You discover you can communicate with animals.
story:
He was in the middle of a famine zone when a man migrated to Israel. He had started fleeing for food the food court brought upon and began getting whipped. Everyone wanted to be there for him. That was until he met a girl he loved who was a tribal girl. They kept tabs on him and kept tabs on him. Eventually he got arrested and he was probed three times and thrusted. Luckily for him, he had a beautiful girlfriend who recently returned from a large hunting trip. He still loves her, and he enjoyed living there. Until the day he began to wonder. I have a secret. I have never trusted someone who really loves me. The first time I truly got around to them I discovered that they have no love to share. After many months, he seems to become obsessed with me. Suddenly, researching and restraining me on some land a few countries

## THE END

### Rerences:

https://huggingface.co/blog/how-to-generate
https://www.kaggle.com/code/ratthachat/writingprompts-gpt2-lm-fine-tune/notebook


